Формулировка лаб4: 

Для датасета с отзывами с маркетплейса на русском языке (https://github.com/sismetanin/rureviews) построить модель для предсказания тональности текста.

1. Использовать как минимум 3 модели машинного обучения, решающие задачу классификации (количество классов определить по анализу датасета)
2. Предсказать тональность при помощи NLP-моделей (deeppavlov, natasha и т.д.) 
3. Определить метрики качества моделей и сравнить полученные результаты, в ячейке markdown представить выводы.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from deeppavlov import build_model, configs
import numpy as np

# Загрузка данных
df = pd.read_csv('datasets/women-clothing-accessories.3-class.balanced.csv', encoding="utf-8", engine='python', sep='\t')
print(df.head())

                                              review sentiment
0  качество плохое пошив ужасный (горловина напер...  negative
1  Товар отдали другому человеку, я не получила п...  negative
2  Ужасная синтетика! Тонкая, ничего общего с пре...  negative
3  товар не пришел, продавец продлил защиту без м...  negative
4      Кофточка голая синтетика, носить не возможно.  negative


In [2]:
# Проверка на наличие пропущенных значений
print(df.isnull().sum())

review       0
sentiment    0
dtype: int64


In [3]:
# Анализ распределения классов
print(df['sentiment'].value_counts())

negative    30000
neautral    30000
positive    30000
Name: sentiment, dtype: int64


In [4]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

# Векторизация текста с использованием TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

Обучение моделей машинного обучения

Модель 1: Наивный Байес

In [5]:
# Обучение модели Наивного Байеса
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# Предсказание на тестовых данных
y_pred_nb = nb_model.predict(X_test_tfidf)

# Оценка качества модели
print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

Naive Bayes Accuracy: 0.7137222222222223
              precision    recall  f1-score   support

    neautral       0.60      0.67      0.63      6060
    negative       0.72      0.63      0.67      5942
    positive       0.85      0.83      0.84      5998

    accuracy                           0.71     18000
   macro avg       0.72      0.71      0.72     18000
weighted avg       0.72      0.71      0.72     18000



Модель 2: Логистическая регрессия

In [6]:
# Обучение модели Логистической регрессии
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_tfidf, y_train)

# Предсказание на тестовых данных
y_pred_lr = lr_model.predict(X_test_tfidf)

# Оценка качества модели
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

Logistic Regression Accuracy: 0.7403333333333333
              precision    recall  f1-score   support

    neautral       0.63      0.65      0.64      6060
    negative       0.73      0.72      0.72      5942
    positive       0.87      0.85      0.86      5998

    accuracy                           0.74     18000
   macro avg       0.74      0.74      0.74     18000
weighted avg       0.74      0.74      0.74     18000



Модель 3: Случайный лес

In [7]:
# Обучение модели Случайного леса
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)

# Предсказание на тестовых данных
y_pred_rf = rf_model.predict(X_test_tfidf)

# Оценка качества модели
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.7185
              precision    recall  f1-score   support

    neautral       0.61      0.64      0.62      6060
    negative       0.73      0.68      0.71      5942
    positive       0.82      0.83      0.83      5998

    accuracy                           0.72     18000
   macro avg       0.72      0.72      0.72     18000
weighted avg       0.72      0.72      0.72     18000



Использование NLP-моделей для предсказания тональности

Модель DeepPavlov

In [8]:
# Загрузка модели DeepPavlov для анализа тональности
sentiment_model = build_model(configs.classifiers.rusentiment_bert, download=True)

# Предсказание тональности для тестовых данных
y_pred_dp = [sentiment_model([text])[0] for text in X_test]

# Оценка качества модели
print("DeepPavlov Accuracy:", accuracy_score(y_test, y_pred_dp))
print(classification_report(y_test, y_pred_dp))

2025-03-15 15:34:14.625 INFO in 'deeppavlov.core.data.utils'['utils'] at line 97: Downloading from http://files.deeppavlov.ai/v1/classifiers/rusentiment_bert/rusentiment_bert_torch.tar.gz to C:\Users\One__\.deeppavlov\models\classifiers\rusentiment_bert_torch.tar.gz
100%|██████████| 1.34G/1.34G [00:33<00:00, 40.0MB/s]
2025-03-15 15:34:48.597 INFO in 'deeppavlov.core.data.utils'['utils'] at line 284: Extracting C:\Users\One__\.deeppavlov\models\classifiers\rusentiment_bert_torch.tar.gz archive into C:\Users\One__\.deeppavlov\models\classifiers\rusentiment_bert_torch
C:\Users\One__\anaconda3\envs\deeppavlov\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

C:\Users\One__\anaconda3\envs\deeppavlov\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\One__\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

DeepPavlov Accuracy: 0.3244444444444444
              precision    recall  f1-score   support

    neautral       0.00      0.00      0.00      6060
    negative       0.58      0.35      0.44      5942
     neutral       0.00      0.00      0.00         0
    positive       0.63      0.62      0.63      5998
        skip       0.00      0.00      0.00         0
      speech       0.00      0.00      0.00         0

    accuracy                           0.32     18000
   macro avg       0.20      0.16      0.18     18000
weighted avg       0.40      0.32      0.35     18000



C:\Users\One__\anaconda3\envs\deeppavlov\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\One__\anaconda3\envs\deeppavlov\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\One__\anaconda3\envs\deeppavlov\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\One__\

Оценка качества моделей и сравнение результатов

In [9]:
# Сравнение результатов
results = {
    'Naive Bayes': accuracy_score(y_test, y_pred_nb),
    'Logistic Regression': accuracy_score(y_test, y_pred_lr),
    'Random Forest': accuracy_score(y_test, y_pred_rf),
    'DeepPavlov': accuracy_score(y_test, y_pred_dp)
}

for model, accuracy in results.items():
    print(f"{model}: {accuracy:.4f}")

Naive Bayes: 0.7137
Logistic Regression: 0.7403
Random Forest: 0.7185
DeepPavlov: 0.3244


Вывод:

1.Модель Logistic Regression показала наилучшую точность среди классических моделей машинного обучения.
2.Random Forest и Naive Bayes показали несколько худшие результаты, но все же остаются достаточно эффективными для данной задачи.
3.Модель DeepPavlov  показала наихудшую точность, ее предсказания при применении к отзывам оказались не столь эффективными.